## Case Study - Majority vote

The goal of this notebook is to explore the performance of GPT-3 in majority vote questions on our 20 question dataset.  Specifically, we want to give a question, five answers, and let GPT-3 aggregate them into one single answer.

1. Imports and functions
2. Simple approach
3. Chain of thought

## 1. Imports and functions

In [4]:
import re
import os
import openai
from time import time, sleep
import textwrap
import pandas as pd

In [3]:
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()


def save_file(filepath, content):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

# change path to where you store your credentials
openai.api_key = open_file('../creds/creds.txt')


def gpt3_completion(prompt, label='gpt3', engine='text-davinci-002', temp=0.5, top_p=1.0, tokens=400, freq_pen=2.0, pres_pen=2.0, stop=['asdfasdf', 'asdasdf']):
    max_retry = 5
    retry = 0
    prompt = prompt.encode(encoding='ASCII', errors='ignore').decode()  # force it to fix any unicode errors
    while True:
        try:
            response = openai.Completion.create(
                engine=engine,
                prompt=prompt,
                temperature=temp,
                max_tokens=tokens,
                top_p=top_p,
                frequency_penalty=freq_pen,
                presence_penalty=pres_pen,
                stop=stop)
            text = response['choices'][0]['text'].strip()
            text = re.sub('\s+', ' ', text)
            return text
        except Exception as oops:
            retry += 1
            if retry >= max_retry:
                return "GPT3 error: %s" % oops
            print('Error communicating with OpenAI:', oops)
            sleep(1)


In [10]:
df_maj = pd.read_csv('majority_questions.csv')

In [11]:
df_maj

,question_number,question,answer_1,answer_2,answer_3,answer_4,answer_5,validation
0,1,What is left in the box?,"The box has a pencil, another 10 pencils and a...",The box has 21 pencils and a strawberry.,"The box has a pencil, a strawberry, and anothe...",The box has 21 pencils and a strawberry.,"The box has 10 pencils, a strawberry, and anot...",The box has 11 pencils and a strawberry.
1,2,What is left in the box?,"The box has a pencil, another 10 pencils and a...",The box has 21 pencils and a strawberry.,"The box has a pencil, a strawberry, and anothe...",The box has 21 pencils and a strawberry.,"The box has 10 pencils, a strawberry, and anot...",The box has 21 pencils and a strawberry.
2,3,Should congress pass the law?,The law should go through,The law should be banned,The law should get the greenlight,The law should not be part of the constitution,The law should be implemented,The law should go through
3,4,Should congress pass the law?,The law should go through,The law should be banned,The law should get the greenlight,The law should not be part of the constitution,The law should not be implemented,The law should not be implemented
4,5,Did the experiment work correctly?,Yes,No,I don't know,Most likely,Probably,Yes
5,6,Did the experiment work correctly?,Yes,No,I don't know,I'm not sure,Probably not,No
6,7,How should I wash my hands?,Use handsoap,Use soap,Use alcohol,Use aloe vera soap,Use shampoo,Soap
7,8,How should I wash my hands?,Use hair shampoo,Use soap,Use shampoo,Use aloe vera soap,Use shampoo,Shampoo
8,9,What should we eat today?,Pizza,Sushi,Pizza or Sushi,Sushi,Hamburger,Sushi
9,10,What should we eat today?,Pizza,Sushi,Pizza,Sushi or Pizza,Hamburger,Pizza


## 2. Simple approach

We'll use the following template:

In the text below we have 5 answers to the same question.

ANSWERS:

- answer_1
- answer_2
- answer_3
- answer_4
- answer_5

What is the answer to "__question__". Count how many times each answer appears, and choose the most frequent.

Also, we will use the function for finding if two answers are equivalent, as we explored in the prior notebook "Case Study - Validation"

In [16]:
## Validation function
def equiv(question, answer_1, answer_2):
    prompt_beginning = """In the following text we have a question and two answers.  Are the two answers the same? Answer with "Yes" or "No" """            
    nl_str = """\n\n"""
    question_str = """QUESTION:\n"""
    answers_str = """ANSWERS: \n"""
    final_answer_cot = """Are they the same answer? Use step by step reasoning and then output only a "Yes" or "No" answer:"""
    prompt = prompt_beginning + nl_str + question_str + question + nl_str + answers_str + "-" + answer_1 + "\n" + "-" + answer_2 + nl_str + final_answer_cot
    final_answer = gpt3_completion(prompt, temp = 0.3)
    if final_answer not in ['Yes','No']:
        prompt_init = """Condense the following sentence into a simple "Yes" or "No" answer"""
        prompt = prompt_init + nl_str + final_answer + nl_str + """ "Yes" or "No" output: """
        final_a = gpt3_completion(prompt)
        final_a.replace('.','').replace(' ','')
        final_answer = final_a
    final_answer.replace('.','').replace(' ','')
    return final_answer
    

In [18]:
temp_list = [0.2, 0.4, 0.6, 0.8, 0.9, 0.95, 1]

In [23]:
prompt_beginning = """In the text below we have 5 answers to the same question."""            
nl_str = """\n\n"""
answers_str = """ANSWERS: \n"""
final_str_1 = """What is the answer to \""""
final_str_2 = """\". Count how many times each answer appears, and choose the most frequent."""


In [ ]:
score_list = []
for temp in temp_list:
    gpt_cot_list = []
    gpt_val_list = []
    print(temp)
    for row in df_maj.iterrows():
        question = row[1]['question']
        answers_concat =  "\n -" + row[1]['answer_1'] + "\n -" + row[1]['answer_2'] + "\n -" + row[1]['answer_3'] + "\n -" + row[1]['answer_4'] + "\n -" + row[1]['answer_5']
        prompt = prompt_beginning + nl_str + answers_str + answers_concat + nl_str + final_str_1 + question + final_str_2
        print(prompt)
        final_answer = gpt3_completion(prompt, temp = temp)
        #print(final_answer)
        #final_answer.replace('.','').replace(' ','')
        print(row[0])
        gpt_cot_list.append(final_answer)
        equiv_string = equiv(question, final_answer, row[1]['validation'])
        if equiv_string not in ['Yes','No']:
            prompt_init = """Condense the following sentence into a simple "Yes" or "No" answer"""
            prompt = prompt_init + nl_str + final_answer + nl_str + """ "Yes" or "No" output: """
            final_a = gpt3_completion(prompt, temp = 0.3)
            final_a.replace('.','').replace(' ','')
            equiv_string = final_a
        gpt_val_list.append(equiv_string)
    col_string_cot = 'CoT_'+ str(temp)
    col_string_val = 'Val_'+ str(temp)
    df_maj[col_string_cot] = gpt_cot_list
    df_maj[col_string_val] = gpt_val_list
    # validate against real answers using the equiv function above 
    score = sum(df_maj[col_string_val]=='Yes')/20
    score_list.append(score) 

In [34]:
df_results = pd.DataFrame(list(zip(temp_list, score_list)), columns =['Temp', 'Score'])
df_results

,Temp,Score
0,0.20,0.50
1,0.40,0.45
2,0.60,0.45
3,0.80,0.55
4,0.90,0.45
5,0.95,0.40
6,1.00,0.60


Not quite a success.. let's see specifically what is happening

In [35]:
df_maj

,question_number,question,answer_1,answer_2,answer_3,answer_4,answer_5,validation,CoT_0.2,Val_0.2,...,CoT_0.6,Val_0.6,CoT_0.8,Val_0.8,CoT_0.9,Val_0.9,CoT_0.95,Val_0.95,CoT_1,Val_1
0,1,What is left in the box?,"The box has a pencil, another 10 pencils and a...",The box has 21 pencils and a strawberry.,"The box has a pencil, a strawberry, and anothe...",The box has 21 pencils and a strawberry.,"The box has 10 pencils, a strawberry, and anot...",The box has 11 pencils and a strawberry.,"The box has a pencil, another 10 pencils and a...",Yes,...,"The box has a pencil, another 10 pencils and a...",No,"The box has a pencil, another 10 pencils and a...",Yes,"The box has a pencil, another 10 pencils and a...",Yes,"The box has a pencil, another 10 pencils and a...",No,"The box has a pencil, another 10 pencils and a...",Yes
1,2,What is left in the box?,"The box has a pencil, another 10 pencils and a...",The box has 21 pencils and a strawberry.,"The box has a pencil, a strawberry, and anothe...",The box has 21 pencils and a strawberry.,"The box has 10 pencils, a strawberry, and anot...",The box has 21 pencils and a strawberry.,"The box has a pencil, another 10 pencils and a...",No,...,"The box has a pencil, another 10 pencils and a...",No,The box has 21 pencils and a strawberry.,Yes,"The box has a pencil, another 10 pencils and a...",No,"The most frequent answer is ""The box has 21 pe...",Yes,"The box has a pencil, another 10 pencils and a...",No
2,3,Should congress pass the law?,The law should go through,The law should be banned,The law should get the greenlight,The law should not be part of the constitution,The law should be implemented,The law should go through,The law should go through.,Yes,...,The law should go through.,Yes,"The answer is ""the law should go through"" with...",Yes,The law should go through.,Yes,"The answer is ""the law should go through.""",Yes,The law should go through.,Yes
3,4,Should congress pass the law?,The law should go through,The law should be banned,The law should get the greenlight,The law should not be part of the constitution,The law should not be implemented,The law should not be implemented,"The answer is ""the law should go through"" with...",No,...,"The answer to ""Should congress pass the law?"" ...",No,The law should go through.,No,The law should go through.,No,"The answer is ""the law should go through.""",No,"The answer is ""should not be part of the const...",No
4,5,Did the experiment work correctly?,Yes,No,I don't know,Most likely,Probably,Yes,-Yes,Yes,...,Yes,Yes,Yes,Yes,Yes,Yes,Most likely,Yes,Yes,Yes
5,6,Did the experiment work correctly?,Yes,No,I don't know,I'm not sure,Probably not,No,"The most frequent answer is ""I don't know.""",No,...,"The most frequent answer is ""I don't know.""",No,-Yes,Yes,"The answer is ""I don't know"" because it appear...",No,"The most frequent answer is ""I don't know.""",No,"The most frequent answer is ""I don't know.""",No
6,7,How should I wash my hands?,Use handsoap,Use soap,Use alcohol,Use aloe vera soap,Use shampoo,Soap,"The answer to ""How should I wash my hands?"" is...",Yes,...,-Use soap,Yes,-Use soap,Yes,"The answer is ""use soap.""",Yes,Use soap.,Yes,"The answer to ""How should I wash my hands?"" th...",Yes
7,8,How should I wash my hands?,Use hair shampoo,Use soap,Use shampoo,Use aloe vera soap,Use shampoo,Shampoo,"The most frequent answer is ""Use shampoo.""",No,...,The answer is to use shampoo.,No,"The most frequent answer is ""Use shampoo"" whic...",No,The answer is to use shampoo.,No,"The most frequent answer is ""use shampoo"" whic...",No,-Use shampoo,No
8,9,What should we eat today?,Pizza,Sushi,Pizza or Sushi,Sushi,Hamburger,Sushi,"Pizza and sushi are both mentioned twice, so i...",No,...,The answer is pizza.,No,"The answer is ""Pizza or Sushi"" because it appe...",No,Pizza – 2 Sushi – 3 Pizza or Sushi – 1 Hamburg...,No,Pizza: 2 Sushi: 3,No,Pizza and Sushi are tied with 2 each.,No
9,10,What should we eat today?,Pizza,Sushi,Pizza,Sushi or Pizza,H

## 3. Chain of thought

In [44]:
prompt_beginning = """In the text below we have 5 answers to the same question."""            
nl_str = """\n\n"""
answers_str = """ANSWERS: \n"""
final_str_1 = """What is the answer to \""""
final_str_2 = """\". Count how many times each answer appears, and choose the most frequent. Reason your answer step-by-step."""
# Notice we added the step by step reason in the last string



In [ ]:
score_list = []
for temp in temp_list:
    gpt_cot_list = []
    gpt_val_list = []
    print(temp)
    for row in df_maj.iterrows():
        question = row[1]['question']
        answers_concat =  "\n -" + row[1]['answer_1'] + "\n -" + row[1]['answer_2'] + "\n -" + row[1]['answer_3'] + "\n -" + row[1]['answer_4'] + "\n -" + row[1]['answer_5']
        prompt = prompt_beginning + nl_str + answers_str + answers_concat + nl_str + final_str_1 + question + final_str_2
        print(prompt)
        final_answer = gpt3_completion(prompt, temp = temp)
        #print(final_answer)
        #final_answer.replace('.','').replace(' ','')
        print(row[0])
        gpt_cot_list.append(final_answer)
        equiv_string = equiv(question, final_answer, row[1]['validation'])
        if equiv_string not in ['Yes','No']:
            prompt_init = """Condense the following sentence into a simple "Yes" or "No" answer"""
            prompt = prompt_init + nl_str + final_answer + nl_str + """ "Yes" or "No" output: """
            final_a = gpt3_completion(prompt, temp = 0.3)
            final_a.replace('.','').replace(' ','')
            equiv_string = final_a
        gpt_val_list.append(equiv_string)
    col_string_cot = 'CoT_'+ str(temp)
    col_string_val = 'Val_'+ str(temp)
    df_maj[col_string_cot] = gpt_cot_list
    df_maj[col_string_val] = gpt_val_list
    # validate against real answers using the equiv function above 
    score = sum(df_maj[col_string_val]=='Yes')/20
    score_list.append(score) 

In [38]:
df_results = pd.DataFrame(list(zip(temp_list, score_list)), columns =['Temp', 'Score'])
df_results

,Temp,Score
0,0.20,0.45
1,0.40,0.35
2,0.60,0.25
3,0.80,0.30
4,0.90,0.25
5,0.95,0.50
6,1.00,0.30


Ouch, that's even worse than before..

In [42]:
pd.set_option('display.max_colwidth', None)


In [43]:
df_maj

,question_number,question,answer_1,answer_2,answer_3,answer_4,answer_5,validation,CoT_0.2,Val_0.2,CoT_0.4,Val_0.4,CoT_0.6,Val_0.6,CoT_0.8,Val_0.8,CoT_0.9,Val_0.9,CoT_0.95,Val_0.95,CoT_1,Val_1
0,1,What is left in the box?,"The box has a pencil, another 10 pencils and a strawberry.",The box has 21 pencils and a strawberry.,"The box has a pencil, a strawberry, and another 10 pencils.",The box has 21 pencils and a strawberry.,"The box has 10 pencils, a strawberry, and another pencil.",The box has 11 pencils and a strawberry.,"The most frequent answer is ""The box has a pencil, another 10 pencils and a strawberry."" This answer appears twice.",Yes,"The answer that appears most frequently is ""The box has 21 pencils and a strawberry."" This answer makes the most sense, because it states that there are exactly 21 items in the box- 1 pencil, 10 other pencils, and 1 strawberry. The other answers either do not state an exact number of items, or they list the items in a different order.",No,"The answer that appears most frequently is ""The box has 21 pencils and a strawberry."" This answer makes the most sense, as it includes all of the items mentioned in the other answers.",No,"The most frequent answer is ""The box has a pencil, another 10 pencils, and a strawberry."" This answer appears twice.",No,"The answer that appears most frequently is ""The box has a pencil, another 10 pencils, and a strawberry.""",No,"The answer that appears most frequently is ""The box has 21 pencils and a strawberry."" This answer makes the most sense, because it includes all of the items mentioned in the other answers.",No,"The most frequent answer is ""The box has 21 pencils and a strawberry."" This answer appears twice.",No
1,2,What is left in the box?,"The box has a pencil, another 10 pencils and a strawberry.",The box has 21 pencils and a strawberry.,"The box has a pencil, a strawberry, and another 10 pencils.",The box has 21 pencils and a strawberry.,"The box has 10 pencils, a strawberry, and another 11 pencils.",The box has 21 pencils and a strawberry.,"The answer that appears most frequently is ""The box has 21 pencils and a strawberry."" This answer makes the most sense, because it includes all of the items mentioned in the other answers.",Yes,"The most frequent answer is ""The box has 21 pencils and a strawberry."" This answer appears twice. The other answers each appear once.",Yes,"The most frequent answer is ""The box has 21 pencils and a strawberry."", which appears twice. The other answers all appear once, so they are less likely to be correct.",Yes,"-The box has a pencil, another 10 pencils and a strawberry. This answer appears once. -The box has 21 pencils and a strawberry. This answer appears twice. -The box has a pencil, a strawberry, and another 10 pencils. This answer appears once. -The box has 21 pencils and a Strawberry. This Answer Appears Twice -The Box Has 10 Pencils, A Strawberry, And Another 11 Pencils."" this Answer Appears Once From the above answers we can see that ""the box has 21 pencils and 1 strawberry"" is the most frequent with appearing 3 times out of 5 answers given . Therefore this is the correct answer to what is left in the box?",Yes,"The most frequent answer is ""the box has 21 pencils and a strawberry."" This answer appears twice, while the other answers appear only once.",Yes,"The most frequent answer is ""The box has 21 pencils and a strawberry."" This answer appears twice. The other answers each appear once.",Yes,The box has 21 pencils and a strawberry.,Yes
2,3,Should congress pass the law?,The law should go through,The law should be banned,The law should get the greenlight,The law should not be part of the constitution,The law should be implemented,The law should go through,"The answer to ""Should congress pass the law?"" is that the law should go through. This is based on the fact that three out of five answers say that the law should either go through, get the greenlight, or be implemented.",Yes,"The most freque

Sometimes it's too verbose and the equiv function gets confused.  Maybe re-writing the prompt so that it is more concise in it's initial answer.

In [47]:
prompt_beginning = """In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below."""            
nl_str = """\n\n"""
answers_str = """ANSWERS: \n"""
final_str_1 = """What is the answer to \""""
final_str_2 = """\". Count how many times each answer appears, reason step-by-step and respond in as few words as possible."""
# Notice we added the step by step reason in the last string



In [48]:
score_list = []
for temp in temp_list:
    gpt_cot_list = []
    gpt_val_list = []
    print(temp)
    for row in df_maj.iterrows():
        question = row[1]['question']
        answers_concat =  "\n -" + row[1]['answer_1'] + "\n -" + row[1]['answer_2'] + "\n -" + row[1]['answer_3'] + "\n -" + row[1]['answer_4'] + "\n -" + row[1]['answer_5']
        prompt = prompt_beginning + nl_str + answers_str + answers_concat + nl_str + final_str_1 + question + final_str_2
        print(prompt)
        final_answer = gpt3_completion(prompt, temp = temp)
        #print(final_answer)
        #final_answer.replace('.','').replace(' ','')
        print(row[0])
        gpt_cot_list.append(final_answer)
        equiv_string = equiv(question, final_answer, row[1]['validation'])
        if equiv_string not in ['Yes','No']:
            prompt_init = """Condense the following sentence into a simple "Yes" or "No" answer"""
            prompt = prompt_init + nl_str + final_answer + nl_str + """ "Yes" or "No" output: """
            final_a = gpt3_completion(prompt, temp = 0.3)
            final_a.replace('.','').replace(' ','')
            equiv_string = final_a
        gpt_val_list.append(equiv_string)
    col_string_cot = 'CoT_'+ str(temp)
    col_string_val = 'Val_'+ str(temp)
    df_maj[col_string_cot] = gpt_cot_list
    df_maj[col_string_val] = gpt_val_list
    # validate against real answers using the equiv function above 
    score = sum(df_maj[col_string_val]=='Yes')/20
    score_list.append(score) 

0.2
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another pencil.

What is the answer to "What is left in the box?". Count how many times each answer appears, reason step-by-step and respond in as few words as possible.
0
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another 11 pencils.


19
0.4
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another pencil.

What is the answer to "What is left in the box?". Count how many times each answer appears, reason step-by-step and respond in as few words as possible.
0
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another 11 pencil

19
0.6
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another pencil.

What is the answer to "What is left in the box?". Count how many times each answer appears, reason step-by-step and respond in as few words as possible.
0
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another 11 pencil

19
0.8
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another pencil.

What is the answer to "What is left in the box?". Count how many times each answer appears, reason step-by-step and respond in as few words as possible.
0
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another 11 pencil

19
0.9
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another pencil.

What is the answer to "What is left in the box?". Count how many times each answer appears, reason step-by-step and respond in as few words as possible.
0
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another 11 pencil

19
0.95
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another pencil.

What is the answer to "What is left in the box?". Count how many times each answer appears, reason step-by-step and respond in as few words as possible.
0
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another 11 penci

19
1
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another pencil.

What is the answer to "What is left in the box?". Count how many times each answer appears, reason step-by-step and respond in as few words as possible.
0
In the text below we have 5 answers to the same question. You will need to choose the popular option in an extremely short answer below.

ANSWERS: 

 -The box has a pencil, another 10 pencils and a strawberry.
 -The box has 21 pencils and a strawberry.
 -The box has a pencil, a strawberry, and another 10 pencils.
 -The box has 21 pencils and a strawberry.
 -The box has 10 pencils, a strawberry, and another 11 pencils.

19


In [49]:
df_results = pd.DataFrame(list(zip(temp_list, score_list)), columns =['Temp', 'Score'])
df_results

,Temp,Score
0,0.20,0.40
1,0.40,0.35
2,0.60,0.45
3,0.80,0.45
4,0.90,0.30
5,0.95,0.40
6,1.00,0.40


In [50]:
df_maj

,question_number,question,answer_1,answer_2,answer_3,answer_4,answer_5,validation,CoT_0.2,Val_0.2,CoT_0.4,Val_0.4,CoT_0.6,Val_0.6,CoT_0.8,Val_0.8,CoT_0.9,Val_0.9,CoT_0.95,Val_0.95,CoT_1,Val_1
0,1,What is left in the box?,"The box has a pencil, another 10 pencils and a strawberry.",The box has 21 pencils and a strawberry.,"The box has a pencil, a strawberry, and another 10 pencils.",The box has 21 pencils and a strawberry.,"The box has 10 pencils, a strawberry, and another pencil.",The box has 11 pencils and a strawberry.,"The box has a pencil, another 10 pencils and a strawberry.",No,"The box has a pencil, another 10 pencils and a strawberry.",No,"The box has a pencil, another 10 pencils and a strawberry.",Yes,"The box has a pencil, another 10 pencils and a strawberry.",No,"The box has a pencil, another 10 pencils and a strawberry.",No,"The box has a pencil, another 10 pencils and a strawberry.",Yes,"The most popular answer is ""The box has a pencil, another 10 pencils, and a strawberry.""",No
1,2,What is left in the box?,"The box has a pencil, another 10 pencils and a strawberry.",The box has 21 pencils and a strawberry.,"The box has a pencil, a strawberry, and another 10 pencils.",The box has 21 pencils and a strawberry.,"The box has 10 pencils, a strawberry, and another 11 pencils.",The box has 21 pencils and a strawberry.,"The box has a pencil, another 10 pencils and a strawberry.",No,"The box has a pencil, another 10 pencils and a strawberry.",No,"-The box has a pencil, another 10 pencils and a strawberry. -The box has 21 pencils and a strawberry. -The box has a pencil, a strawberry, and another 10 pencils. -The box has 21 pencils and a strawberry. -The box has 10 pencils, a strawberry, and another 11pencils. There are 3 popular answers: -""the boxhas21pencilsandastrawberry"" -""theboxhasapencilandanother10pencilsandastrawberry"" -""theboxhasapencila strawberrryandanother10pencillls""",No,"The box has a pencil, another 10 pencils and a strawberry.",No,"The box has a pencil, another 10 pencils and a strawberry.",No,"The box has a pencil, another 10 pencils and a strawberry.",No,"The popular answer is ""The box has a pencil, another 10 pencils and a strawberry."" This answer appears twice.",Yes
2,3,Should congress pass the law?,The law should go through,The law should be banned,The law should get the greenlight,The law should not be part of the constitution,The law should be implemented,The law should go through,The law should go through.,Yes,The law should go through.,Yes,"The answer should be ""The law should go through"" because it is the most popular option.",Yes,"The answer to ""Should congress pass the law?"" is that the law should go through.",Yes,"The most popular answer is ""the law should go through"" with 2 occurrences.",Yes,"The answer that appears the most is ""The law should go through.""",Yes,The law should go through.,Yes
3,4,Should congress pass the law?,The law should go through,The law should be banned,The law should get the greenlight,The law should not be part of the constitution,The law should not be implemented,The law should not be implemented,"The answer with the most appearances is ""the law should go through.""",No,"The answer is ""the law should go through."" This appears twice in the text.",No,The law should go through.,No,"The answer with the most appearances is ""The law should go through.""",No,"The answer with the most votes is ""the law should go through.""",No,The law should go through.,No,The law should go through.,No
4,5,Did the experiment work correctly?,Yes,No,I don't know,Most likely,Probably,Yes,Most likely,Yes,Most likely,No,Most likely.,Yes,Most likely.,No,I don't know.,No,Yes,Yes,The experiment worked correctly. -Yes,Yes
5,6,Did the experiment work correctly?,Yes,No,I don't know,I'm not sure,Probably not,No,"The answer is ""No."" Three people said ""No,"" one person said ""Probably not,"" and one person said ""I'm not sure.""",No,Yes,Yes,"The most popular answer is ""I don't know"

Let's see.  These are too tricky:

- Godfather -> It is actually correct if we consider it is a trilogy
- Stock market (both), too varied responses I guess
- How was the movie. the bad one.  "not great" is confusing I guess
- Experiment one (No) --> equiv function is messed up unfortunately
- Negative of law should go through is incorrect, strange mistake.


## 4. Conclusion

I will park this experiment as a failure.  My expectation was that the reconciliation could be done in an efficient way, like >90% accuracy.  Even being benevolent with the observations above, this would only give 70-80% accuracy.
It is very possible that prompt engineering solves a lot of this, so might re-visit this notebook at some point (although with GPT-4 on the horizon, maybe not)